In [ ]:
from hstrader import HsTrader
from hstrader.models import Tick, Event, Resolution
import plotly.graph_objects as go
import pandas as pd
from datetime import timedelta
import os
from dotenv import load_dotenv
import logging
import asyncio
from psar import PSAR
from IPython.display import display, clear_output



In [ ]:
psar = PSAR()
# Enable logging
logging.basicConfig(level=logging.INFO)

# Load environment variables
load_dotenv()

# Get the CLIENT_ID and CLIENT_SECRET from the environment variables
id = os.getenv('CLIENT_ID')
secret = os.getenv('CLIENT_SECRET')

# Initialize the HsTrader client with the client ID and secret
client = HsTrader(id, secret)

### Plot Candlestick Chart with PSAR Indicator (Historical Market Data): 

In [ ]:
# Retrieve market data for any forex symbol by specifying its name in the get_symbol function
symbol = client.get_symbol('ETHXBT')
data = client.get_market_history(symbol=symbol.id, resolution=Resolution.M1)

# Create a DataFrame from the retrieved data
df = pd.DataFrame([bar.model_dump() for bar in data])
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)


In [10]:
# Calculate PSAR values
df['PSAR'] = psar.calc_psar_batch(df['high'].tolist(), df['low'].tolist())

# Add supporting data
df['EP'] = psar.ep_list
df['AF'] = psar.af_list
df['Trend'] = psar.trend_list
df.head()

,open,high,low,close,volume,PSAR,EP,AF,Trend
time,,,,,,,,,
2024-08-19 03:12:00+00:00,80192.8,80194.7,80179.1,80180.4,448.866485,NaN,80194.7,0.02,NaN
2024-08-19 03:13:00+00:00,80180.4,80225.6,80180.4,80193.9,2407.465823,NaN,80225.6,0.02,NaN
2024-08-19 03:14:00+00:00,80193.9,80217.0,80186.4,80204.2,2289.831109,80179.1,80225.6,0.02,1.0
2024-08-19 03:15:00+00:00,80204.2,80204.9,80149.9,80149.9,2017.325623,80217.0,80149.9,0.02,0.0
2024-08-19 03:16:00+00:00,80149.9,80150.0,80149.9,80149.9,1870.152003,80217.0,80149.9,0.02,0.0


In [ ]:

# Separate PSAR values based on trend
psar_bull = df.loc[df['Trend'] == 1]['PSAR']
psar_bear = df.loc[df['Trend'] == 0]['PSAR']

# Create figure
fig = go.Figure()

# Add candlestick chart
fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name='Candlestick'
    ))

# Add PSAR bull scatter
fig.add_trace(go.Scatter(x=psar_bull.index, y=psar_bull, mode='markers', 
                         name='Up Trend', marker=dict(color='green', size=2)))

# Add PSAR bear scatter
fig.add_trace(go.Scatter(x=psar_bear.index, y=psar_bear, mode='markers', 
                         name='Down Trend', marker=dict(color='red', size=2)))

# Update layout
fig.update_layout(
    title=f'{symbol.symbol} Price and Parabolic SAR',
    xaxis_title='Date',
    yaxis_title='Price',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    autosize=True,
    
)

# Show figure
fig.show()

### Example Plot

Below is a static image of Parabolic SAR applied to historical data:
![psar plot](img/psar.png)

### Plot Real-Time Candlestick Chart with PSAR Indicator :
If you’re interested in backtesting and strategy optimization, the code above utilizes the PSAR indicator with historical data. However, if you want real-time data for live trading decisions, you can run the code below to incorporate the indicator in live market conditions.

In [ ]:
from hstrader import HsTrader
from hstrader.models import Tick, Event, Resolution
import plotly.graph_objects as go
import pandas as pd
from datetime import timedelta
import os
from dotenv import load_dotenv
import logging
import asyncio
from psar import PSAR
from IPython.display import display, clear_output
import nest_asyncio

In [ ]:
psar = PSAR()
# Enable logging
logging.basicConfig(level=logging.INFO)

# Load environment variables
load_dotenv()

# Get the CLIENT_ID and CLIENT_SECRET from the environment variables
id = os.getenv('CLIENT_ID')
secret = os.getenv('CLIENT_SECRET')

# Initialize the HsTrader client with the client ID and secret
client = HsTrader(id, secret)

In [ ]:
# Retrieve market data for any forex symbol by specifying its name in the get_symbol function
symbol = client.get_symbol('Ethereum')
data = client.get_market_history(symbol=symbol.id, resolution=Resolution.M1)

# Create a DataFrame from the retrieved data
df = pd.DataFrame([bar.model_dump() for bar in data])
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)

# Update the DataFrame with PSAR trend values
df['psar'] = psar.calc_psar_batch(df['high'].tolist(), df['low'].tolist())
df['Trend']=psar.trend_list
psar_bull = df.loc[df['Trend'] == 1]['psar']
psar_bear = df.loc[df['Trend'] == 0]['psar']

In [ ]:
# Setup the initial plot
fig = go.FigureWidget()
candlestick = go.Candlestick(x=df.index, open=df['open'], high=df['high'], low=df['low'], close=df['close'], name='Candlestick')
fig.add_trace(candlestick)
# Add PSAR bull scatter
fig.add_trace(go.Scatter(x=psar_bull.index, y=psar_bull, mode='markers', name='Up Trend', marker=dict(color='green', size=2)))

# Add PSAR bear scatter
fig.add_trace(go.Scatter(x=psar_bear.index, y=psar_bear, mode='markers', name='Down Trend', marker=dict(color='red', size=2)))

# Suppress automatic display
pass

In [ ]:
# Prepare data dictionary
CANDLE_INTERVAL = timedelta(minutes=1)
data = {
    'x': list(df.index),
    'open': list(df['open']),
    'high': list(df['high']),
    'low': list(df['low']),
    'close': list(df['close'])
}

In [ ]:
@client.subscribe(Event.MARKET)
async def on_market(tick: Tick):
    global data, df
    try:
        if tick.symbol_id == symbol.id:
            tick_time = pd.to_datetime(tick.time)
            if not data['x']:
                data['x'].append(tick_time)
                data['open'].append(data['close'][-1])
                data['high'].append(tick.bid)
                data['low'].append(tick.bid)
                data['close'].append(tick.bid)
            elif tick_time >= data['x'][-1] + CANDLE_INTERVAL:
                data['x'].append(tick_time)
                data['open'].append(data['close'][-1])
                data['high'].append(tick.bid)
                data['low'].append(tick.bid)
                data['close'].append(tick.bid)
            else:
                data['low'][-1] = min(tick.bid, data['low'][-1])
                data['high'][-1] = max(tick.bid, data['high'][-1])
                data['close'][-1] = tick.bid

            df = pd.DataFrame({
                'time': data['x'],
                'open': data['open'],
                'high': data['high'],
                'low': data['low'],
                'close': data['close']
            }).set_index('time')

            # Calculate PSAR and trend list
            df['psar'] = psar.calc_psar_batch(df['high'].tolist(), df['low'].tolist())
            df['Trend']= psar.trend_list
    except Exception as e:
        logging.error(f"Error in on_market: {e}")
        await client.refresh_token()

In [ ]:
async def update_plot():
    while True:
        await asyncio.sleep(1)
        clear_output(wait=True)
        psar_bull = df.loc[df['Trend'] == 1]['psar']
        psar_bear = df.loc[df['Trend'] == 0]['psar']
        with fig.batch_update():
            fig.data[0].x = data['x']
            fig.data[0].open = data['open']
            fig.data[0].high = data['high']
            fig.data[0].low = data['low']
            fig.data[0].close = data['close']
            fig.data[1].x = psar_bull.index
            fig.data[1].y = psar_bull
            fig.data[2].x = psar_bear.index
            fig.data[2].y = psar_bear
        display(fig)

In [ ]:
nest_asyncio.apply()
loop=asyncio.get_event_loop()
loop.run_until_complete(asyncio.gather(client.start_async(), update_plot()))